# Development notebook. No experiments or results here. For dev purposes




In [1]:
run init.ipynb

Done.


# Experiment on the best imputation technics in the case of single disribution in the bayesian framework 


First: multi-distribution - INDEPENDANT 

Second: Single-distribution - Which imputation method ? 

Third: NAM: Which config is best between encoding or imputation ?

Fourth: NAM: Which config is best between using indicator Variables or not ? 

Fifth: 

In [2]:
"""  Parameters of the experiment """
approach = 'multi_distributions'

range_dataset_name = ['circles', 'moons', 'blobs']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.1, 0.25, 0.5]
range_missingness_pattern = [1, 3, 4, 5]

missing_data_handling = 'without'
imputation_method = 'without'
use_missing_indicator_variables = False

In [4]:
df = create_df()
df

,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,moons,32,multi_distributions,without,without,None,8000,0.25,5,MNAR,None,True,True,True,0,0.0785,NaN,0.2480,0.2100,-0.3750,0.4070,0.1960,0.1420,0.1420,0.5040,0.5930,0.8580,0.4960,30,0.2,NaN,NaN
1,moons,20,multi_distributions,without,without,None,8000,0.25,1,MCAR,0.3,True,False,None,None,None,NaN,0.3750,0.2080,-0.2380,0.3370,0.3870,0.1500,0.1500,0.6450,0.6630,0.8500,0.3550,30,0.2,NaN,NaN
2,moons,18,multi_distributions,without,without,None,8000,0.25,1,MCAR,0.1,True,False,None,None,None,NaN,0.2890,0.1340,-0.4150,0.2140,0.3150,0.0970,0.0970,0.5370,0.7860,0.9030,0.4630,30,0.2,NaN,NaN
3,moons,27,multi_distributions,without,without,None,8000,0.25,3,MAR,0.2,True,False,True,None,None,NaN,0.2990,0.0980,-0.4310,0.1550,0.3460,0.0720,0.0720,0.5560,0.8450,0.9280,0.4440,30,0.2,NaN,NaN
4,moons,9,multi_distributions,without,without,None,8000,0.10,3,MAR,0,True,False,True,None,None,NaN,0.1760,0.0410,-0.4530,0.1760,0.1760,0.0230,0.0230,0.6600,0.8240,0.9770,0.3400,30,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,blobs,13,nam,encoding,without,True,8000,0.10,2,MAR,0.3,True,True,True,None,None,0.9552,0.9711,0.8296,0.8284,0.7100,0.9998,0.9983,0.9983,0.9690,0.2900,0.0017,0.0310,30,0.2,NaN,NaN
108,blobs,5,nam,encoding,without,True,8000,0.10,1,MCAR,0.2,True,False,None,None,None,0.9804,0.9781,0.8759,0.8720,0.7798,0.9999,0.9993,0.9993,0.9763,0.2202,0.0007,0.0237,30,0.2,NaN,NaN
109,blobs,14,nam,encoding,without,True,8000,0.10,3,MAR,0,True,False,True,None,None,1.0000,0.9993,0.9974,0.9971,0.9955,0.9999,0.9993,0.9993,0.9993,0.0045,0.0007,0.0007,30,0.2,NaN,NaN
110,blobs,22,nam,encoding,without,True,8000,0.10,5,MNAR,None,True,True,True,0,0.476875,1.0000,0.9992,0.9967,0.9962,0.9933,1.0000,1.0000,1.0000,0.9992,0.0067,0.0000,0.0008,30,0.2,NaN,NaN


In [6]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0



save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:

        for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

            for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):


                if check_experiment_already_done(df, 
                                                  dataset_name=dataset_name, 
                                                  imbalance_ratio=imbalance_ratio, 
                                                  imputation_method=imputation_method,
                                                  missingness_pattern=missingness_pattern, 
                                                  use_missing_indicator_variables=use_missing_indicator_variables,
                                                  ratio_of_missing_values=ratio_of_missing_values, 
                                                  ratio_missing_per_class=ratio_missing_per_class):


                    print("Experiment already done.")
                    continue

                print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                    missing_data_handling,
                                                                                                                                                      imputation_method, 
                                                                                                                                                      imbalance_ratio,
                                                                                                                                                      missingness_pattern,
                                                                                                                                                      ratio_of_missing_values, 
                                                                                                                                                      ratio_missing_per_class))
                
                start_time = time()
                
                try:

                    dataset = DatasetGenerator(dataset_name=dataset_name, 
                                            num_samples=num_samples, 
                                            imbalance_ratio=imbalance_ratio, 
                                            missing_data_handling=missing_data_handling,
                                            imputation_method=imputation_method,
                                            verbosity=verbosity)


                    # Creation of the missingness
                    dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                         ratio_of_missing_values=ratio_of_missing_values, 
                                                         ratio_missing_per_class=ratio_missing_per_class)

                    dataset.split_test_train()

                    #X_all = dataset.X_test


                    #print(np.argwhere( (X_all[:,0]>=2.5) | (X_all[:,0] <= -2.5) | (X_all[:,1]>=2.5) | (X_all[:,1] <= -2.5) ).squeeze())

                    # Create the experiments
                    exp = Experiments(dataset_name, 
                                      dataset=dataset, 
                                      purpose='classification', 
                                      approach = approach, 
                                      previous_experiment=None, 
                                      save_experiment=save, 
                                      verbosity=verbosity, 
                                      debug=False, 
                                      proportion_train=PROPORTION_TRAIN, 
                                      resolution=RESOLUTION, 
                                      bandwidth=BANDWIDTH,
                                      random_state=RANDOM_STATE)


                    # Estimate the distributions 
                    exp.fit()

                    # Estimate the distributions 
                    exp.predict()

                    # Plot results
                    #exp.plot()


                    end_time = time()

                    hours, rest = divmod(end_time - start_time, 3600)
                    minutes, seconds = divmod(rest, 60)
                    print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                except:
                    print("/!\. Error!\n")


112
circles - imputation method: without missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.0 ratio_per_class: None
Doing experiment 49!
Done (0h 1m and 20.14s).

112
circles - imputation method: without missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.1 ratio_per_class: None
Doing experiment 50!
Done (0h 1m and 12.36s).

112
circles - imputation method: without missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.2 ratio_per_class: None
Doing experiment 51!
Done (0h 1m and 17.61s).

112
circles - imputation method: without missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.3 ratio_per_class: None
Doing experiment 52!
Done (0h 1m and 18.66s).

112
circles - imputation method: without missing_data_handling: without imbalance: 0.1 pattern: 2 ratio_missing:0.0 ratio_per_class: None
Doing experiment 53!
Done (0h 1m and 18.86s).

112
circles - imputation method: without missing_data_handling: without imb